In [7]:
# Import the necessary libraries

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib import cm
import matplotlib as mpl
import imageio

from shapely.vectorized import contains

In [8]:
# Load the raster file, shapefile, and district coordinates

mp_raster = pd.read_csv("Export/all_raster_data_MP.csv")
mp_raster["Date"] = pd.to_datetime(mp_raster["Date"], format="%Y-%m-%d")

india_map = gpd.read_file("gadm41_IND_shp/gadm41_IND_1.shp")
madhya_pradesh = india_map[india_map['NAME_1'] == 'Madhya Pradesh']

district_coords = pd.read_csv("Export/district_coords_MP.csv")
gdf = gpd.GeoDataFrame(district_coords, geometry=gpd.points_from_xy(district_coords['Longitude'], district_coords['Latitude']), crs="EPSG:4326")

In [9]:
# Set up matplotlib parameters and custom colormap

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 10
mpl.rcParams['axes.titleweight'] = 'bold'

r_palette = ["#2b83ba", "#92c5de", "#d1e5f0", "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d"]
custom_cmap = LinearSegmentedColormap.from_list("r_volatility", r_palette)
norm = Normalize(vmin=0, vmax=1)

In [10]:
# Generate 3D Volatility Surface GIF and save separate frames

output_dir = "CAR_Frames_MP_3D"
os.makedirs(output_dir, exist_ok=True)

times = mp_raster['Date'].drop_duplicates().sort_values()
latitudes = np.sort(mp_raster['x'].unique())
longitudes = np.sort(mp_raster['y'].unique()) 

frames_3d = []

for i, t in enumerate(times):
    dft = mp_raster[mp_raster['Date'] == t]
    
    grid = dft.pivot(index='y', columns='x', values='Volatility')
    Z = grid.values

    Lon, Lat = np.meshgrid(grid.columns.values, grid.index.values)

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    surf = ax.plot_surface(Lon, Lat, Z, cmap=custom_cmap, norm=norm,
                           linewidth=0, antialiased=True)

    mappable = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    mappable.set_array([])
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, pad=0.1, aspect=10)
    cbar.set_label("Volatility", weight='bold')

    ax.set_title(f"Soybean Volatility Surface - {t.strftime('%Y-%m')}")
    ax.set_xlim(Lon.min(), Lon.max())
    ax.set_ylim(Lat.min(), Lat.max())
    ax.set_zlim(0, 1)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Volatility")

    frame_path = os.path.join(output_dir, f"vol_3d_{t.strftime('%Y-%m')}.png")
    plt.savefig(frame_path, dpi=150, bbox_inches='tight', pad_inches=0.1)
    plt.close()

    image = imageio.imread(frame_path)
    frames_3d.append(image)

imageio.mimsave("Soybean_price_VolatilitySurface_3D.gif", frames_3d, fps=3, loop=0)
print("3D Volatility GIF saved as 'Soybean_price_VolatilitySurface_3D.gif'")

C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\4132782442.py:43: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_path)


3D Volatility GIF saved as 'Soybean_price_VolatilitySurface_3D.gif'


In [11]:
# Generate 2D Volatility Surface GIF and save separate frames

output_dir = "CAR_Frames_MP_2D"
os.makedirs(output_dir, exist_ok=True)
frames_2d = []

x_unique = np.sort(mp_raster['x'].unique())
y_unique = np.sort(mp_raster['y'].unique())
x_grid, y_grid = np.meshgrid(x_unique, y_unique, indexing='xy')

mp_polygon = madhya_pradesh.geometry.unary_union
mp_mask = contains(mp_polygon, x_grid, y_grid)

times = mp_raster['Date'].drop_duplicates().sort_values()

for i, t in enumerate(times):
    dft = mp_raster[mp_raster['Date'] == t]
    grid = dft.pivot(index='y', columns='x', values='Volatility')
    Z = grid.values

    Z_masked = np.full_like(Z, np.nan)
    Z_masked[mp_mask] = Z[mp_mask]

    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(Z_masked, origin='lower',
                   extent=[x_unique.min(), x_unique.max(), y_unique.min(), y_unique.max()],
                   cmap=custom_cmap, norm=norm, aspect='auto')

    cs = ax.contour(x_unique, y_unique, Z_masked, levels=10, colors='k', linewidths=0.7)
    ax.clabel(cs, inline=True, fontsize=6, fmt="%.2f")

    madhya_pradesh.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5)
    gdf.plot(ax=ax, color='white', edgecolor='black', markersize=40)
    for _, row in gdf.iterrows():
        ax.text(row.geometry.x, row.geometry.y + 0.1, row['District'], fontsize=8, ha='center', va='bottom')

    ax.set_title(f"Soybean Volatility - {t.strftime('%Y-%m')}", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, pad=0.03)
    cbar.set_label("Volatility", weight='bold')
    plt.tight_layout()
    frame_path = os.path.join(output_dir, f"vol_2d_{t.strftime('%Y-%m')}.png")
    plt.savefig(frame_path, dpi=150, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    image = imageio.imread(frame_path)
    frames_2d.append(image)

imageio.mimsave("Soybean_price_VolatilitySurface_2D.gif", frames_2d, fps=3, loop=0)
print("2D Surface Volatility GIF saved as 'Soybean_price_VolatilitySurface_2D.gif'")

C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\2289694587.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mp_polygon = madhya_pradesh.geometry.unary_union
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\2289694587.py:12: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mp_mask = contains(mp_polygon, x_grid, y_grid)
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\2289694587.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_path)
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\2289694587.py:46: DeprecationWarning: Starting with ImageIO v3 the behavior o

2D Surface Volatility GIF saved as 'Soybean_price_VolatilitySurface_2D.gif'


In [12]:
# Generate combined 2D and 3D Volatility Surface GIF and save separate frames

output_dir = "CAR_Frames_MP_Combined"
os.makedirs(output_dir, exist_ok=True)
frames_combined = []

x_unique = np.sort(mp_raster['x'].unique())
y_unique = np.sort(mp_raster['y'].unique())
x_grid, y_grid = np.meshgrid(x_unique, y_unique, indexing='xy')

grid_points = np.column_stack([x_grid.ravel(), y_grid.ravel()])
mp_polygon = madhya_pradesh.geometry.unary_union
mp_mask_flat = contains(mp_polygon, grid_points[:, 0], grid_points[:, 1])
mp_mask = mp_mask_flat.reshape(x_grid.shape)

times = mp_raster['Date'].drop_duplicates().sort_values()

for i, t in enumerate(times):
    dft = mp_raster[mp_raster['Date'] == t]
    grid = dft.pivot(index='y', columns='x', values='Volatility')
    Z = grid.values
    Z_masked = np.full_like(Z, np.nan)
    Z_masked[mp_mask] = Z[mp_mask]

    Lon, Lat = np.meshgrid(grid.columns.values, grid.index.values)

    fig = plt.figure(figsize=(16, 8))
    gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.2)

    ax2d = fig.add_subplot(gs[0, 0])
    im = ax2d.imshow(Z_masked, origin='lower',
                     extent=[x_unique.min(), x_unique.max(), y_unique.min(), y_unique.max()],
                     cmap=custom_cmap, norm=norm, aspect='auto')
    
    CS = ax2d.contour(Lon, Lat, Z_masked, levels=10, colors='k', linewidths=0.7)
    ax2d.clabel(CS, inline=True, fontsize=6, fmt="%.2f")
    
    madhya_pradesh.boundary.plot(ax=ax2d, edgecolor='black', linewidth=1.5)
    gdf.plot(ax=ax2d, color='white', edgecolor='black', markersize=40)
    for _, row in gdf.iterrows():
        ax2d.text(row.geometry.x, row.geometry.y + 0.1, row['District'], fontsize=8, ha='center', va='bottom')
    ax2d.set_xlabel("Longitude", fontsize=14)
    ax2d.set_ylabel("Latitude", fontsize=14)
    ax2d.tick_params(axis='both', labelsize=10)

    ax3d = fig.add_subplot(gs[0, 1], projection='3d')
    surf = ax3d.plot_surface(Lon, Lat, Z, cmap=custom_cmap, norm=norm,
                             linewidth=0, antialiased=True)
    ax3d.set_xlim(Lon.min(), Lon.max())
    ax3d.set_ylim(Lat.min(), Lat.max())
    ax3d.set_zlim(0, 1)
    ax3d.set_xlabel("Longitude", fontsize=14)
    ax3d.set_ylabel("Latitude", fontsize=14)
    ax3d.set_zlabel("Volatility", fontsize=14)
    ax3d.tick_params(axis='both', labelsize=10)

    fig.suptitle(f"Soybean Volatility Surface - {t.strftime('%Y-%m')}", fontsize=16, weight='bold')

    cbar_ax = fig.add_subplot(gs[0, 2])
    mappable = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    mappable.set_array([])
    cbar = plt.colorbar(mappable, cax=cbar_ax, orientation='vertical', shrink=0.5)
    cbar.set_label("Volatility", weight='bold', fontsize=14)
    cbar.ax.tick_params(labelsize=10)

    frame_path = os.path.join(output_dir, f"vol_combined_{t.strftime('%Y-%m')}.png")
    plt.savefig(frame_path, dpi=150, bbox_inches='tight')
    plt.close()

    image = imageio.imread(frame_path)
    frames_combined.append(image)

imageio.mimsave("Soybean_price_VolatilitySurface_2D_3D.gif", frames_combined, fps=3, loop=0)
print("2D+3D Combined Volatility GIF saved as 'Soybean_price_VolatilitySurface_2D_3D.gif'")

C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\1172777614.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mp_polygon = madhya_pradesh.geometry.unary_union
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\1172777614.py:13: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mp_mask_flat = contains(mp_polygon, grid_points[:, 0], grid_points[:, 1])
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\1172777614.py:70: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_path)
C:\Users\Abbin\AppData\Local\Temp\ipykernel_30168\1172777614.py:70: DeprecationWarning: Starting wit

2D+3D Combined Volatility GIF saved as 'Soybean_price_VolatilitySurface_2D_3D.gif'
